# Housing Prices, Take II (because the first try failed epically lol)

In [28]:
%matplotlib inline

#Importing Modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold

In [10]:
# Reading Data
df = pd.read_csv('train.csv')

# Feature Engineering

In [11]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [13]:
#Separating test and train datasets, dropping unnecessary columns
local_train, local_test = train_test_split(df,test_size=0.2,random_state=123)

local_train_y = local_train['SalePrice']
local_train_x = local_train.drop(['SalePrice', 'Id', 'MSSubClass'],axis=1)
local_test_y = local_train['SalePrice']
local_test_x = local_train.drop(['SalePrice', 'Id', 'MSSubClass'],axis=1)


In [14]:
# get all quantitative data

local_train_x_quant = local_train_x

for col in local_train_x.columns:
    if local_train_x[col].dtype == 'O':
        local_train_x_quant = local_train_x_quant.drop([col],axis=1)

local_test_x_quant = local_test_x

for col in local_test_x.columns:
    if local_test_x[col].dtype == 'O':
        local_test_x_quant = local_test_x_quant.drop([col],axis=1)

In [15]:
local_test_x_quant.head()

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
318,90.0,9900,7,5,1993,1993,256.0,987,0,360,...,656,340,60,144,0,0,0,0,4,2009
580,NaN,14585,6,6,1960,1987,85.0,594,219,331,...,572,216,110,0,0,0,0,0,6,2007
961,NaN,12227,6,7,1977,1995,424.0,896,0,434,...,619,550,282,0,0,0,0,0,7,2008
78,72.0,10778,4,5,1968,1968,0.0,0,0,1768,...,0,0,0,0,0,0,0,0,4,2010
5,85.0,14115,5,5,1993,1995,0.0,732,0,64,...,480,40,30,0,320,0,0,700,10,2009


In [19]:
#changing NAs to means
local_train_x_quant = local_train_x_quant.fillna(local_train_x_quant.mean())

False

In [10]:
#replace NaN with mean for that column

num_cols = len(local_test_x_quant.columns)
num_rows = local_test_x_quant.iloc[:,0].size

for c in range(num_cols):
    curr_col = local_test_x_quant.iloc[:,c]
    if curr_col.isnull().any():
        curr_mean = curr_col.mean()
    for r in range(num_rows):
        if np.isnan(curr_col.iloc[r]):
            curr_col.iloc[r] = curr_mean
    local_test_x_quant.iloc[:,c] = curr_col
    
local_test_x_quant = local_test_x_quant.fillna(local_test_x_quant.mean())
local_test_x_quant


,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
318,90.00000,9900,7,5,1993,1993,256.0,987,0,360,...,656,340,60,144,0,0,0,0,4,2009
580,69.58427,14585,6,6,1960,1987,85.0,594,219,331,...,572,216,110,0,0,0,0,0,6,2007
961,69.58427,12227,6,7,1977,1995,424.0,896,0,434,...,619,550,282,0,0,0,0,0,7,2008
78,72.00000,10778,4,5,1968,1968,0.0,0,0,1768,...,0,0,0,0,0,0,0,0,4,2010
5,85.00000,14115,5,5,1993,1995,0.0,732,0,64,...,480,40,30,0,320,0,0,700,10,2009
793,76.00000,9158,8,5,2007,2007,140.0,0,0,1496,...,474,168,130,0,0,0,0,0,6,2007
1425,80.00000,10721,6,6,1959,1959,243.0,0,0,1252,...,528,0,39,0,0,0,0,0,10,2008
1392,68.00000,7838,5,5,1967,1967,0.0,769,0,95,...,288,175,144,0,0,0,0,0,12,2006
585,88.00000,11443,8,5,2005,2006,208.0,1460,0,408,...,880,326,66,0,0,0,0,0,3,2006
876,94.00000,25286,4,5,1963,1963,0.0,633,0,431,...,648,0,0,0,0,0,0,0,1,2007


# Actual models

In [8]:
# #Extract columns (numerical) we care about
# #df_num = df.drop["MSZoning","Street","Alley","LotShape","LandContour","Utilities","LotConfig","LandSlope","Neighborhood","Condition1","Condition2","BldgType","HouseStyle","YearBuilt","YearRemodAdd","RoofStyle","RoofMatl","Exterior1st","Exterior2nd","MasVnrType","ExterQual","ExterCond","Foundation","BsmtQual","BsmtCond",axis = 1, inplace = True]
# df_num = df[["MSZoning","LandSlope","Neighborhood","Condition1","HouseStyle","RoofMatl","KitchenQual","CentralAir","PoolQC","SaleCondition","LotArea","OverallQual","OverallCond","YearBuilt","YearRemodAdd","TotalBsmtSF","1stFlrSF","2ndFlrSF","GrLivArea","BedroomAbvGr","TotRmsAbvGrd","Fireplaces","GarageCars","GarageArea","SalePrice"]]
# #One-Hot-Encoding for categorical columns
# df_only = pd.get_dummies(df_num, columns = ["MSZoning","LandSlope","Neighborhood","Condition1","HouseStyle","RoofMatl","KitchenQual","CentralAir","PoolQC","SaleCondition"], drop_first = True)
# df_only.head(3)

In [9]:
# local_train, local_test = train_test_split(df_only, test_size = 0.2, random_state = 123)

In [20]:
local_train.shape

(1168, 81)

In [21]:
local_test.shape

(292, 81)

In [22]:
#Look at correlations
local_train.corr()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
Id,1.000000,-0.011194,-0.003452,-0.020261,-0.022174,0.018106,-0.008903,-0.003244,-0.065327,-0.021955,...,-0.006439,0.000684,-0.012787,-0.049447,0.009806,0.063541,-0.004687,0.013757,0.005972,-0.009764
MSSubClass,-0.011194,1.000000,-0.383137,-0.140527,0.010195,-0.034672,0.000951,0.012973,0.007641,-0.076540,...,-0.000007,-0.017439,-0.007933,-0.044193,-0.035758,0.009318,-0.005926,0.008051,-0.038677,-0.101455
LotFrontage,-0.003452,-0.383137,1.000000,0.421713,0.268685,-0.052905,0.123528,0.095788,0.216385,0.253866,...,0.075299,0.187686,0.014309,0.082454,0.052242,0.240701,0.011928,-0.010349,0.010975,0.362332
LotArea,-0.020261,-0.140527,0.421713,1.000000,0.109849,-0.008596,0.020954,-0.004920,0.077488,0.224165,...,0.174330,0.083060,-0.031974,0.024868,0.070452,0.093495,0.036294,0.001899,0.002563,0.261746
OverallQual,-0.022174,0.010195,0.268685,0.109849,1.000000,-0.075525,0.573105,0.545806,0.419570,0.261590,...,0.236756,0.302142,-0.110442,0.038212,0.041904,0.072232,-0.029436,0.073212,-0.023196,0.788993
OverallCond,0.018106,-0.034672,-0.052905,-0.008596,-0.075525,1.000000,-0.369359,0.102497,-0.123010,-0.050076,...,0.022753,-0.034670,0.067677,0.012844,0.043522,-0.002770,0.065616,-0.004710,0.047418,-0.071699
YearBuilt,-0.008903,0.000951,0.123528,0.020954,0.573105,-0.369359,1.000000,0.574465,0.327571,0.268097,...,0.206932,0.179828,-0.379291,0.037985,-0.039151,0.006435,-0.032050,-0.010302,-0.021627,0.524452
YearRemodAdd,-0.003244,0.012973,0.095788,-0.004920,0.545806,0.102497,0.574465,1.000000,0.177020,0.151615,...,0.202749,0.201625,-0.178187,0.038480,-0.043483,0.006024,-0.011946,-0.006233,0.034781,0.498253
MasVnrArea,-0.065327,0.007641,0.216385,0.077488,0.419570,-0.123010,0.327571,0.177020,1.000000,0.279630,...,0.149679,0.129393,-0.109717,0.023445,0.042041,0.013451,-0.030787,-0.019857,0.011640,0.473472
BsmtFinSF1,-0.021955,-0.076540,0.253866,0.224165,0.261590,-0.050076,0.268097,0.151615,0.279630,1.000000,...,0.199392,0.124736,-0.111352,0.025097,0.066678,0.153372,0.006882,-0.042554,0.022737,0.407679


### Observations
GarageCars and Garage Area seem to be practically the same (dependent?)

SalePrice seems to be strongly correlated (|correlation| > 0.5) with

- OverallQual, YearBuilt, TotalBsmtSF, 1stFlrSF, GrLivArea, TotRmsAbvGrd, GarageCars/GarageArea

In [23]:
local_train_y.shape

(1168,)

In [24]:
local_train_x_quant.shape

(1168, 35)

In [25]:
#The Model
clf = sm.OLS(local_train_y, local_train_x_quant)
np.asarray(local_train_x_quant)
#result = clf.fit()
#result.summary()
# preds = clf.fit().predict(local_test_x_quant) 
# preds #predicted prices

array([[  9.00000000e+01,   9.90000000e+03,   7.00000000e+00, ...,
          0.00000000e+00,   4.00000000e+00,   2.00900000e+03],
       [  6.95842697e+01,   1.45850000e+04,   6.00000000e+00, ...,
          0.00000000e+00,   6.00000000e+00,   2.00700000e+03],
       [  6.95842697e+01,   1.22270000e+04,   6.00000000e+00, ...,
          0.00000000e+00,   7.00000000e+00,   2.00800000e+03],
       ..., 
       [  6.95842697e+01,   2.07810000e+04,   7.00000000e+00, ...,
          0.00000000e+00,   6.00000000e+00,   2.00600000e+03],
       [  7.00000000e+01,   8.44500000e+03,   5.00000000e+00, ...,
          0.00000000e+00,   3.00000000e+00,   2.00900000e+03],
       [  6.00000000e+01,   6.00000000e+03,   6.00000000e+00, ...,
          0.00000000e+00,   3.00000000e+00,   2.00700000e+03]])

In [27]:
#RMSE of Linear Model
log_actual_price = np.log(local_test_y.values)
log_pred_price = np.log(local_train_y.values)
diffs_squared = (log_actual_price - log_pred_price) ** 2
n = len(local_test_y)
rmse = np.sqrt(np.sum(diffs_squared) / n)
rmse

0.0